## 1. Caching Prompt Embeding

In [ ]:
from src.train_nabla_r2d3_diffsplat_pas import get_diffSlat_model as get_diffSlat_model_pas
from peft import LoraConfig, BOFTConfig
from peft.utils import get_peft_model_state_dict,load_peft_weights
from extensions.diffusers_diffsplat import PixArtSigmaMVPipeline
from transformers import T5EncoderModel, T5Tokenizer

import einops
import os
import torch
from tqdm import tqdm

In [ ]:
import logging    
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

model_dict = get_diffSlat_model_pas(logger,load_text_encoder=True)

In [4]:
pipeline: PixArtSigmaMVPipeline = model_dict["pipeline"]
device = "cuda:0"
pipeline = pipeline.to(device)
# pipeline = pipeline.to("cpu")

### 1.1 Cache positive prompt Embedding( GObj-83k prompt dataset)

In [5]:
import os

save_dir = 'output/Cached_embedding/pas_embedding_G-obj-83k'

file_path = 'src/nablagflow/alignment/assets/G-obj-83k.txt'
### output embedding cache path
### input prompt path 
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

with open(file_path, 'r') as file:
    lines = file.readlines()

prompts = [line.strip() for line in lines]  # Remove any leading/trailing whitespace characters

# print(lines)

In [6]:
!cp  {file_path} {save_dir}

In [ ]:


# for prompt in tqdm(prompts):
idx = 0
batch_size = 100
# pbar = tqdm( len(prompts)//batch_size)
pbar = tqdm(range(0, len(prompts)))
# total=100
# save_idx = 0
cache_every = 1000


prompt_embeds_list = []
prompt_attention_masks_list = []
negative_prompt_attention_mask =None
negative_prompt_embeds = None
prompts_list=[]

half_precision=True 
with torch.autocast("cuda", torch.bfloat16 if half_precision else torch.float32):
    with torch.inference_mode():
        reach_end = False
        
        while True:
            upper_bound = idx + batch_size
            if upper_bound >= len(prompts):
                upper_bound = len(prompts)
                reach_end = True
            if idx > len(prompts):
                break
            pbar.set_description(f"processing {idx}-{upper_bound}")
            prompt = prompts[idx:upper_bound]
            prompt_embeds, prompt_attention_mask, negative_prompt_embeds, negative_prompt_attention_mask = pipeline.encode_prompt(
                prompt=prompt,
                do_classifier_free_guidance= True,
                # negative_prompt= "",
                device= device,
                clean_caption=True,
            )
            prompt_embeds_list.append(prompt_embeds.cpu())
            prompt_attention_masks_list.append(prompt_attention_mask.cpu())
            
            idx += batch_size
            pbar.update(batch_size)
            
            prompts_list.append(prompt)
            
            if (idx) %cache_every == 0 or reach_end:
                prompt_embed_all = torch.cat(prompt_embeds_list,dim=0)
                prompt_attention_masks_all = torch.cat(prompt_attention_masks_list,dim=0)
                a =einops.rearrange(prompt_embed_all, 'b n d -> (b n) d')
                b =einops.rearrange(prompt_attention_masks_all, 'b n -> (b n)')
                selected = a[b==1]
                name = f'{idx-cache_every +1:05d}-{idx:05d}'
                if reach_end:
                    final_start = len(prompts) - len(prompts)%cache_every +1
                    name = f'{final_start:05d}-{upper_bound:05d}'
                torch.save(selected, os.path.join(save_dir,f'./{name}prompt_embeds.pt'))
                torch.save(b,os.path.join(save_dir,f"./{name}prompt_attention_masks.pt"))
                # torch.save(prompts_list, './prompts_list.pt')
                # print("Saved")
                pbar.set_description(f"Saved {name}")
                prompt_embeds_list = []
                prompt_attention_masks_list = []
                torch.cuda.empty_cache()
            if reach_end:
                break
                
                
                



processing 24300-24400:  29%|██▉       | 24300/83296 [57:04<2:15:27,  7.26it/s]

### 1.2 Cache Negative Prompt Embedding

In [ ]:
with torch.no_grad():

    with torch.autocast("cuda", torch.bfloat16 if half_precision else torch.float32):
        prompt_embeds, prompt_attention_mask, negative_prompt_embeds, negative_prompt_attention_mask = pipeline.encode_prompt(
            prompt='',
            do_classifier_free_guidance= True,
            # negative_prompt= "",
            device= "cuda",
            clean_caption=True,
        )
    torch.save(prompt_embeds, os.path.join(save_dir,'negative_prompt_embeds.pt'))
    torch.save(prompt_attention_mask, os.path.join(save_dir,'negative_prompt_attention_mask.pt'))
